In [38]:
import numpy as np
import pandas as pd
import statsmodels.formula.api as sm

In [39]:
bike = pd.read_csv('Bikeshare.csv', index_col='Unnamed: 0')
bike.head()

,season,mnth,day,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,bikers
1,1,Jan,1,0,0,6,0,clear,0.24,0.2879,0.81,0.0,3,13,16
2,1,Jan,1,1,0,6,0,clear,0.22,0.2727,0.80,0.0,8,32,40
3,1,Jan,1,2,0,6,0,clear,0.22,0.2727,0.80,0.0,5,27,32
4,1,Jan,1,3,0,6,0,clear,0.24,0.2879,0.75,0.0,3,10,13
5,1,Jan,1,4,0,6,0,clear,0.24,0.2879,0.75,0.0,0,1,1


In [40]:
bike.drop(columns=['day', 'hr', 'holiday', 'weekday'], inplace=True)
bike.head()

,season,mnth,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,bikers
1,1,Jan,0,clear,0.24,0.2879,0.81,0.0,3,13,16
2,1,Jan,0,clear,0.22,0.2727,0.80,0.0,8,32,40
3,1,Jan,0,clear,0.22,0.2727,0.80,0.0,5,27,32
4,1,Jan,0,clear,0.24,0.2879,0.75,0.0,3,10,13
5,1,Jan,0,clear,0.24,0.2879,0.75,0.0,0,1,1


In [41]:
bike.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8645 entries, 1 to 8645
Data columns (total 11 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   season      8645 non-null   int64  
 1   mnth        8645 non-null   object 
 2   workingday  8645 non-null   int64  
 3   weathersit  8645 non-null   object 
 4   temp        8645 non-null   float64
 5   atemp       8645 non-null   float64
 6   hum         8645 non-null   float64
 7   windspeed   8645 non-null   float64
 8   casual      8645 non-null   int64  
 9   registered  8645 non-null   int64  
 10  bikers      8645 non-null   int64  
dtypes: float64(4), int64(5), object(2)
memory usage: 810.5+ KB


In [42]:
from sklearn.preprocessing import scale
for i in bike.drop(columns=['season', 'mnth', 'workingday', 'weathersit', 'casual', 'registered', 'bikers']).columns:
    bike[i+'_s'] = scale(bike[i], with_mean=True, with_std=True)
bike.head()

,season,mnth,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,bikers,temp_s,atemp_s,hum_s,windspeed_s
1,1,Jan,0,clear,0.24,0.2879,0.81,0.0,3,13,16,-1.258356,-1.024616,0.848628,-1.551923
2,1,Jan,0,clear,0.22,0.2727,0.80,0.0,8,32,40,-1.359400,-1.110613,0.797681,-1.551923
3,1,Jan,0,clear,0.22,0.2727,0.80,0.0,5,27,32,-1.359400,-1.110613,0.797681,-1.551923
4,1,Jan,0,clear,0.24,0.2879,0.75,0.0,3,10,13,-1.258356,-1.024616,0.542945,-1.551923
5,1,Jan,0,clear,0.24,0.2879,0.75,0.0,0,1,1,-1.258356,-1.024616,0.542945,-1.551923


In [43]:
from statsmodels.tools.tools import add_constant
from statsmodels.stats.outliers_influence import variance_inflation_factor
bikeQuant = add_constant(bike.drop(columns=['season', 'mnth', 'workingday', 'weathersit', 'casual', 'registered', 'bikers']))
vif = pd.DataFrame()
vif['Features'] = bikeQuant.columns
vif['VIF'] = [variance_inflation_factor(bikeQuant.values, i) for i in range (len(bikeQuant.columns))]

c:\Users\Kavita manoj\AppData\Local\Programs\Python\Python310\lib\site-packages\statsmodels\regression\linear_model.py:1752: RuntimeWarning: divide by zero encountered in double_scalars
  return 1 - self.ssr/self.centered_tss
c:\Users\Kavita manoj\AppData\Local\Programs\Python\Python310\lib\site-packages\statsmodels\stats\outliers_influence.py:195: RuntimeWarning: divide by zero encountered in double_scalars
  vif = 1. / (1. - r_squared_i)


In [44]:
vif

,Features,VIF
0,const,0.000000e+00
1,temp,3.898327e+04
2,atemp,4.373529e+04
3,hum,1.963972e+03
4,windspeed,1.027414e+06
5,temp_s,inf
6,atemp_s,5.973673e+04
7,hum_s,6.597375e+04
8,windspeed_s,2.009504e+04


In [45]:
sm.ols('bikers~temp_s+atemp_s+hum_s+workingday+windspeed_s', data=bike).fit().summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 bikers   R-squared:                       0.279
Model:                            OLS   Adj. R-squared:                  0.278
Method:                 Least Squares   F-statistic:                     667.5
Date:                Sat, 15 Apr 2023   Prob (F-statistic):               0.00
Time:                        16:16:45   Log-Likelihood:                -53183.
No. Observations:                8645   AIC:                         1.064e+05
Df Residuals:                    8639   BIC:                         1.064e+05
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
===============================================================================
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
Intercept     144.8281      2.177     66.530      0.000     140.561     149.095
temp_s         19.0772     10.233      1.864      0.062      -0.981      39.136
atemp_s        40.2587     10.241      3.931      0.000      20.184      60.333
hum_s         -35.8515      1.270    -28.224      0.000     -38.341     -33.362
workingday     -1.5117      2.634     -0.574      0.566      -6.675       3.652
windspeed_s     3.9621      1.328      2.983      0.003       1.358       6.566
==============================================================================
Omnibus:                     1145.100   Durbin-Watson:                   0.441
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             1726.655
Skew:                           0.961   Prob(JB):                         0.00
Kurtosis:                       4.048   Cond. No.                         16.7
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [46]:
sm.ols('bikers~atemp_s+hum_s+windspeed_s', data=bike).fit().summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 bikers   R-squared:                       0.278
Model:                            OLS   Adj. R-squared:                  0.278
Method:                 Least Squares   F-statistic:                     1111.
Date:                Sat, 15 Apr 2023   Prob (F-statistic):               0.00
Time:                        16:16:45   Log-Likelihood:                -53185.
No. Observations:                8645   AIC:                         1.064e+05
Df Residuals:                    8641   BIC:                         1.064e+05
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
===============================================================================
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
Intercept     143.7944      1.223    117.608      0.000     141.398     146.191
atemp_s        59.1764      1.225     48.315      0.000      56.776      61.577
hum_s         -35.9900      1.268    -28.378      0.000     -38.476     -33.504
windspeed_s     4.6765      1.269      3.685      0.000       2.189       7.164
==============================================================================
Omnibus:                     1135.158   Durbin-Watson:                   0.442
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             1706.146
Skew:                           0.956   Prob(JB):                         0.00
Kurtosis:                       4.041   Cond. No.                         1.32
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [47]:
ohe =pd.get_dummies(data=bike, columns=['weathersit'])

In [48]:
sm.ols('bikers~atemp_s+hum_s+windspeed_s+weathersit', data=bike).fit().summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 bikers   R-squared:                       0.280
Model:                            OLS   Adj. R-squared:                  0.280
Method:                 Least Squares   F-statistic:                     560.1
Date:                Sat, 15 Apr 2023   Prob (F-statistic):               0.00
Time:                        16:16:45   Log-Likelihood:                -53175.
No. Observations:                8645   AIC:                         1.064e+05
Df Residuals:                    8638   BIC:                         1.064e+05
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
=================================================================================================
                                    coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------
Intercept                       141.4842      1.561     90.617      0.000     138.424     144.545
weathersit[T.cloudy/misty]       11.2630      2.968      3.795      0.000       5.445      17.081
weathersit[T.heavy rain/snow]    33.2613    113.636      0.293      0.770    -189.493     256.016
weathersit[T.light rain/snow]    -6.4561      4.761     -1.356      0.175     -15.789       2.877
atemp_s                          59.3047      1.228     48.281      0.000      56.897      61.713
hum_s                           -36.2823      1.408    -25.765      0.000     -39.043     -33.522
windspeed_s                       4.9289      1.291      3.817      0.000       2.398       7.460
==============================================================================
Omnibus:                     1142.123   Durbin-Watson:                   0.444
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             1721.749
Skew:                           0.959   Prob(JB):                         0.00
Kurtosis:                       4.050   Cond. No.                         105.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [49]:
ohe.columns= ['season', 'mnth', 'workingday', 'temp', 'atemp', 'hum', 'windspeed','casual', 'registered', 'bikers', 'temp_s', 'atemp_s', 'hum_s','windspeed_s', 'weathersit_clear', 'weathersit_cloudyOrmisty','weathersit_heavy_rainOrsnow', 'weathersit_light_rainOrsnow']

In [50]:
sm.ols('bikers~atemp_s+hum_s+windspeed_s+weathersit_cloudyOrmisty', data=ohe).fit().summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 bikers   R-squared:                       0.280
Model:                            OLS   Adj. R-squared:                  0.280
Method:                 Least Squares   F-statistic:                     839.6
Date:                Sat, 15 Apr 2023   Prob (F-statistic):               0.00
Time:                        16:16:45   Log-Likelihood:                -53176.
No. Observations:                8645   AIC:                         1.064e+05
Df Residuals:                    8640   BIC:                         1.064e+05
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
============================================================================================
                               coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------
Intercept                  140.6228      1.424     98.755      0.000     137.832     143.414
atemp_s                     59.4212      1.225     48.513      0.000      57.020      61.822
hum_s                      -37.0394      1.290    -28.716      0.000     -39.568     -34.511
windspeed_s                  4.6050      1.268      3.632      0.000       2.119       7.091
weathersit_cloudyOrmisty    12.3620      2.853      4.333      0.000       6.769      17.955
==============================================================================
Omnibus:                     1145.782   Durbin-Watson:                   0.444
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             1731.004
Skew:                           0.960   Prob(JB):                         0.00
Kurtosis:                       4.058   Cond. No.                         2.75
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""